# MLP-100 Numpy Model Extraction

Loads the Dataset and a Model saved from training notebook then translates into Numpy implementation. The dataset is modified with the accuracy from Numpy model (which should match the original model). Then the model and the dataset is exported as sqlite3 databases for implementation in C.  

**NOTE:** The dataset exported by the training notebook may have incorrect predicted index due to several iterations of model training and not updating the dataset. We'll re-run the predictions here and update the predicted index in the dataset.

# Environment Setup

In [1]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm.auto import tqdm


# We don't need GPU for this, not training
#if torch.cuda.is_available():
#    device = torch.device('cuda')
#else:
#    device = torch.device('cpu')

#print('Using PyTorch version:', torch.__version__, ' Device:', device)
print('Using PyTorch version:', torch.__version__)

Using PyTorch version: 2.0.1


# Load and Validate torch.nn.Module

## Define Model

**NOTE:** Always copy the following cell from the training notebook.

In [2]:
# Define an MLP with single hiddend layer with 12 units and ReLU activation.
class MLP100(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLP100, self).__init__()
        # Save parameters
        self.input_size = input_size
        self.num_classes = num_classes
        self.debug = False    # can be used to activate debugging features
        # Define layers
        self.fc1 = nn.Linear(input_size, 100)   # 100 hidden units
        self.fc1_drop = nn.Dropout(0.2)         # drop-out for faster training, has no effect on inference
        self.fc2 = nn.Linear(100, 100)          # 100 hidden units
        self.fc2_drop = nn.Dropout(0.2)         # drop-out for faster training, has no effect on inference
        self.fc3 = nn.Linear(100, num_classes)  # output layer

    # Expects a batch of 1-D tensor
    # Dimension of x: (batch-size, input_size)
    def forward(self, x):
        x = F.relu(self.fc1(x))   # pass through the first hidden layer
        x = self.fc1_drop(x)      
        x = F.relu(self.fc2(x))   # pass through the second hidden layer
        x = self.fc2_drop(x)      
        x = self.fc3(x)           # pass through the output layer
        return x

## Load Saved Model

In [3]:
!ls -ltr ./saved/
print('')

# Load saved model dictionary
model_path = './saved/trained_mlp100-98.36p.pt'
model_dict = torch.load(model_path)
print(model_dict.keys())

# Parse the values for easier use
Accuracy = model_dict['accuracy']
Correct_count = model_dict['correct_count']
Hparam = model_dict['Hparam']
Model_state_dict = model_dict['state_dict']
Model_perf = f'Model Performance:   accuracy: {Accuracy:.2f}%   correct_count: {Correct_count}'  # to be used later
print('Hparam:', Hparam)

# move all weights to cpu
for key in Model_state_dict: 
    Model_state_dict[key] = Model_state_dict[key].to('cpu')

# Instantiate the model
model_pt = MLP100(Hparam['input_size'], Hparam['num_classes'])
model_pt.load_state_dict(Model_state_dict)
print(model_pt)

total 156204
-rw-rw-r-- 1 makabir makabir   360783 Jun 20 17:42 trained_mlp100-98.11p.pt
-rw-rw-r-- 1 makabir makabir   360783 Jun 20 18:07 trained_mlp100-98.34p.pt
-rw-rw-r-- 1 makabir makabir   360783 Jun 21 13:55 trained_mlp100.pt
-rw-rw-r-- 1 makabir makabir 70809849 Jun 21 13:55 test_dataset.pt
-rw-rw-r-- 1 makabir makabir   360783 Jun 21 13:57 trained_mlp100-98.36p.pt
-rw-r--r-- 1 makabir makabir    16384 Jun 21 15:02 model.s3db
-rw-r--r-- 1 makabir makabir  1818624 Jun 21 16:12 model-mlp100.s3db
-rw-r--r-- 1 makabir makabir  1818624 Jun 21 19:52 trained_mlp100-98.36p.s3db
-rw-r--r-- 1 makabir makabir 82194432 Jun 21 19:53 mnist_test_data-98.36p.s3db
-rw-r--r-- 1 makabir makabir  1818624 Jun 21 19:57 trained_mlp100.s3db
-rw-r--r-- 1 makabir makabir    12288 Jun 21 19:57 mnist_test_data.s3db

dict_keys(['accuracy', 'correct_count', 'Hparam', 'state_dict'])
Hparam: {'input_size': 784, 'num_classes': 10}
MLP100(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (fc1_dr

## Load Saved Dataset

In [4]:
# Prints a dataset item
def print_dataitem(item):
    mstr = f"label: {item[0]}, label_index: {item[1]}, predicted_index: {item[2]}, feature_length: {item[3]},"
    mstr2 = f"feature_vector size: {len(item[4])}"
    print(mstr, mstr2)

    
# Load the test dataset
ds_path = './saved/test_dataset.pt'
DS_loaded = torch.load(ds_path)
for key in DS_loaded:
    if key != 'dataset':
        print(f'{key}:', DS_loaded[key])

# Show an item summary
item = DS_loaded['dataset'][0]
print('item-> ', end='')
print_dataitem(item)

label_dict: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}
dataset_schema: (label, label_index, predicted_index, feature_length, feature_vector)
item-> label: 7, label_index: 7, predicted_index: 7, feature_length: 784, feature_vector size: 784


## Validate The Loaded Model

In [5]:
# find most likely label index for each element
def get_likely_index(tensor):
    # convert to tensor from numpy if needed
    if not torch.is_tensor(tensor):
        tensor = torch.from_numpy(tensor)
    return tensor.argmax(dim=-1)


# Given an item form the test_dataset, returns an example for predict() function
# numpytype: set it to True to return numpy nd-array
def make_example(data_item, numpytype=False):
    feature = torch.tensor(data_item[4])
    if numpytype: feature = feature.detach().numpy()
    return feature


# test prediction from dataset item.
# ptmodel: set it to True for the PyTorch model
def predict(example, model=None, ptmodel=False):  # example: feature_vector
    if ptmodel: model.eval()    # set the pytorch model to evaluation mode
    # Use the model to predict the label of the image
    feature = example
    if ptmodel: feature = feature.unsqueeze(0)    # add the batch dimension for the pytorch model
    output = model(feature)
    pred = get_likely_index(output)
    if ptmodel: pred = pred[0]    # removing batch index
    return pred.item()


# Test predict()
item = DS_loaded['dataset'][0]
example = make_example(item)
pred = predict(example, model=model_pt, ptmodel=True)
print('pred:',pred)
print_dataitem(item)

pred: 7
label: 7, label_index: 7, predicted_index: 7, feature_length: 784, feature_vector size: 784


In [6]:
# Validate the Given model on the whole dataset
# ptmodel: set it to True for the PyTorch model
def validateModel(model=None, ptmodel=False, numpytype=False):
    dataset = DS_loaded['dataset']
    expect_miss = 0      # keeps track of no. of mismatche between prediction in dataset vs model prediction
    total_count = 0
    correct_count = 0
    for item in tqdm(dataset):
        lbl, lbl_index, pred_index, *_ = item
        example = make_example(item, numpytype=numpytype)
        pred = predict(example, model=model, ptmodel=ptmodel)
        if pred != pred_index: expect_miss += 1    # prediction does not match prediction in dataset
        if pred == lbl_index: correct_count += 1   # prediction matched the actual label-index
        total_count += 1
    # Compute and print statistics
    accuracy = (100.0 * correct_count) / total_count
    print(f'Validation accuracy: {accuracy:.2f}%   correct_count: {correct_count}   expected-miss: {expect_miss}   total_count: {total_count}')
    return accuracy, correct_count, expect_miss, total_count

            
# Validate the loaded model
validateModel(model_pt, ptmodel=True)
print('Expected', Model_perf)

  0%|          | 0/10000 [00:00<?, ?it/s]

Validation accuracy: 98.36%   correct_count: 9836   expected-miss: 0   total_count: 10000
Expected Model Performance:   accuracy: 98.36%   correct_count: 9836


# Implementation Using torch.tensor Operations

In [7]:
# Extract the weights as torch.tensors
for key in Model_state_dict:
    print(f'{key:10}:', Model_state_dict[key].size())

fc1_weight_pt = Model_state_dict['fc1.weight']
fc1_bias_pt = Model_state_dict['fc1.bias']
fc2_weight_pt = Model_state_dict['fc2.weight']
fc2_bias_pt = Model_state_dict['fc2.bias']
fc3_weight_pt = Model_state_dict['fc3.weight']
fc3_bias_pt = Model_state_dict['fc3.bias']

fc1.weight: torch.Size([100, 784])
fc1.bias  : torch.Size([100])
fc2.weight: torch.Size([100, 100])
fc2.bias  : torch.Size([100])
fc3.weight: torch.Size([10, 100])
fc3.bias  : torch.Size([10])


In [8]:
# Define the model using pytorch tensor operations.
# Input interface is the same as the 
def tensorModel(features):
    x1 = fc1_weight_pt @ features + fc1_bias_pt
    fc1_out = F.relu(x1)
    x2 = fc2_weight_pt @ fc1_out + fc2_bias_pt
    fc2_out = F.relu(x2)
    fc3_out = fc3_weight_pt @ fc2_out + fc3_bias_pt
    return fc3_out


# Test this model
item = DS_loaded['dataset'][0]
example = make_example(item)
pred = predict(example, model=tensorModel, ptmodel=False)
print('pred:',pred)
print_dataitem(item)

pred: 7
label: 7, label_index: 7, predicted_index: 7, feature_length: 784, feature_vector size: 784


In [9]:
# Validate the tensor operation based model
validateModel(tensorModel, ptmodel=False)
print('Expected', Model_perf)

  0%|          | 0/10000 [00:00<?, ?it/s]

Validation accuracy: 98.36%   correct_count: 9836   expected-miss: 0   total_count: 10000
Expected Model Performance:   accuracy: 98.36%   correct_count: 9836


# Implement Using Numpy Matrix Operations

In [10]:
# Copy weights as numpy ndarray
fc1_weight_np = fc1_weight_pt.detach().numpy()
fc1_bias_np   = fc1_bias_pt.detach().numpy()
fc2_weight_np = fc2_weight_pt.detach().numpy()
fc2_bias_np   = fc2_bias_pt.detach().numpy()
fc3_weight_np = fc3_weight_pt.detach().numpy()
fc3_bias_np   = fc3_bias_pt.detach().numpy()

print('fc1_weight_np:', fc1_weight_np.shape)

fc1_weight_np: (100, 784)


In [11]:
# Relu on numpy array
def npReLU(np_arr):
    return np.maximum(0, np_arr)


# Define the model using numpy matrix operations
# Input interface is the same as the 
def numpyModel(features):
    x1 = fc1_weight_np @ features + fc1_bias_np
    fc1_out = npReLU(x1)
    x2 = fc2_weight_np @ fc1_out + fc2_bias_np
    fc2_out = npReLU(x2)
    fc3_out = fc3_weight_np @ fc2_out + fc3_bias_np
    return fc3_out


# Test this model
item = DS_loaded['dataset'][0]
example = make_example(item, numpytype=True)
pred = predict(example, model=numpyModel, ptmodel=False)
print('pred:',pred)
print_dataitem(item)

pred: 7
label: 7, label_index: 7, predicted_index: 7, feature_length: 784, feature_vector size: 784


In [12]:
# Validate the tensor operation based model
validateModel(numpyModel, ptmodel=False, numpytype=True)
print('Expected', Model_perf)

  0%|          | 0/10000 [00:00<?, ?it/s]

Validation accuracy: 98.36%   correct_count: 9836   expected-miss: 0   total_count: 10000
Expected Model Performance:   accuracy: 98.36%   correct_count: 9836


# Update the dataset with the Numpy Model Predicted index

In [13]:
enum_iter = tqdm( enumerate(DS_loaded['dataset']), total=len(DS_loaded['dataset']) )
fix_count = 0
for index, item in enum_iter:
    # Make prediction using Numpy model
    example = make_example(item, numpytype=True)
    pred = predict(example, model=numpyModel, ptmodel=False)
    # Check and fix the predicted_index in the dataset
    if pred!=item[2]:
        DS_loaded['dataset'][index][2] = pred
        fix_count += 1

print(f'INFO: Fixed {fix_count} predicted_index in the dataset')

  0%|          | 0/10000 [00:00<?, ?it/s]

INFO: Fixed 0 predicted_index in the dataset


# Export Numpy Model as sqlite3 DB

## Create the database file

In [14]:
import sqlite3
import os


# Create the sqlite3 database file, overwrite if exist
# Check if the database file already exists
def createDB(db_path, overwrite=False):
    # Check overwrite conditions
    file_exist = os.path.exists(db_path)
    if file_exist and overwrite==False:
        print(f'ERROR: {db_path} exist, try overwrite=True')
        return False
    # remove old file if exist
    if file_exist:
        print('WARN: Overwriting existing file', db_path)
        os.remove(db_path)
    # Connect to the SQLite database file with overwrite option
    conn = sqlite3.connect(db_path, isolation_level=None)
    conn.close()
    return True


DB_path = './saved/trained_mlp100.s3db'
createDB(DB_path, overwrite=True)
!ls -ltr saved/

WARN: Overwriting existing file ./saved/trained_mlp100.s3db
total 154428
-rw-rw-r-- 1 makabir makabir   360783 Jun 20 17:42 trained_mlp100-98.11p.pt
-rw-rw-r-- 1 makabir makabir   360783 Jun 20 18:07 trained_mlp100-98.34p.pt
-rw-rw-r-- 1 makabir makabir   360783 Jun 21 13:55 trained_mlp100.pt
-rw-rw-r-- 1 makabir makabir 70809849 Jun 21 13:55 test_dataset.pt
-rw-rw-r-- 1 makabir makabir   360783 Jun 21 13:57 trained_mlp100-98.36p.pt
-rw-r--r-- 1 makabir makabir    16384 Jun 21 15:02 model.s3db
-rw-r--r-- 1 makabir makabir  1818624 Jun 21 16:12 model-mlp100.s3db
-rw-r--r-- 1 makabir makabir  1818624 Jun 21 19:52 trained_mlp100-98.36p.s3db
-rw-r--r-- 1 makabir makabir 82194432 Jun 21 19:53 mnist_test_data-98.36p.s3db
-rw-r--r-- 1 makabir makabir    12288 Jun 21 19:57 mnist_test_data.s3db
-rw-r--r-- 1 makabir makabir        0 Jun 21 19:58 trained_mlp100.s3db


## Database Utility Functions

In [15]:
# Returns a list of table names in the database file
def getTableNames(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Fetch the table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_names = cursor.fetchall()
    table_names = [name[0] for name in table_names]  # make a list to return
    conn.close()
    return table_names


# Returns a list of column names of the specified table
def getColNames(database_filename, table_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(database_filename)
    cursor = conn.cursor()

    # Fetch the column names
    cursor.execute(f"PRAGMA table_info({table_name})")
    results = cursor.fetchall()
    column_names = [result[1] for result in results]  # Extract the column names from the query results

    # Close the connection and return 
    conn.close()
    return column_names

    
# returns all records of a given table
def getRecords(db_path, table_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # Fetch all records from the table
    cursor.execute(f"SELECT * FROM {table_name}")
    records = cursor.fetchall()
    return records


# Deletes a table from the database
def dropTable(db_path, table_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # Drop the table if it exists
    cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
    # Commit the changes and close the connection
    conn.commit()
    conn.close()
    

# Deletes the rows of the given table
def deleteRows(db_path, table_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # Delete all rows from the table
    cursor.execute(f"DELETE FROM {table_name}")
    conn.commit()
    conn.close()


# Checks if a table exist
def existTable(db_path, table_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # Check if the table exists
    cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
    result = cursor.fetchone()
    if result is None: exist = False
    else: exist = True
    # Commit the changes and close the connection and return result
    conn.commit()
    conn.close()
    return exist

## Write the header table

The header table contains information about the rest of the database.

In [16]:
# Creates the header table
def createHeaderTable(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # Create the table
    query_str = '''CREATE TABLE IF NOT EXISTS Header (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        key TEXT,
                        value NUMERIC,
                        description TEXT
                    )'''
    cursor.execute(query_str)
    # Commit the changes and close the connection
    conn.commit()
    conn.close()


# Create the table and check 
createHeaderTable(DB_path)
table_names = getTableNames(DB_path)
print(table_names)

['Header', 'sqlite_sequence']


In [17]:
#dropTable(DB_path, 'Header')
#print(getTableNames(DB_path))

In [18]:
# Inserts a record into the Header table
def insertHeaderRecord(cursor, record):
    key, value, description = record  # this serves as a soft check for the record
    # Insert the record into the table
    cursor.execute('''INSERT INTO Header (key, value, description)
                      VALUES (?, ?, ?)''', (key, value, description))


# Inserts a list of records into the Header table
# recordFunc: A function that takes a cursor and a record and inserts into the table (insertHeaderRecord)
# record_list: list of tuples (key, target, description)
def insertRecordList(db_path, recordFunc, record_list):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # Insert the records
    for record in record_list:
        recordFunc(cursor, record)
    # Commit the changes and close the connection
    conn.commit()
    conn.close()


# Call the function to insert a record
insertRecordList(DB_path, insertHeaderRecord, [('example_key', 'example_value', 'example_description')])
getRecords(DB_path, 'Header')

[(1, 'example_key', 'example_value', 'example_description')]

In [19]:
# Define the records as a list
Fc1w_table = 'FC1_Weight_T'
Fc1b_table = 'FC1_Bias_T'
Fc2w_table = 'FC2_Weight_T'
Fc2b_table = 'FC2_Bias_T'
Fc3w_table = 'FC3_Weight_T'
Fc3b_table = 'FC3_Bias_T'
Hparam_table = 'Hparam_T'

header_records = [
    ('name', 'MLP-100', ''),
    ('architecture', '784-FC:100-FC:100-10', 'It is an MLP with 2 hidden layer with 100 units in each layer with ReLU activation. Trained on MNIST dataset (output layer with 10 units).'),
    ('accuracy', Accuracy, 'Accuracy% of the trained model on the test dataset.'),
    ('correct_count', Correct_count, 'Number of correct predictions by the trained model on the test dataset.'),
    
    ('Hparam.table',   Hparam_table, 'This is the name of the table that contains different parameters of the model.'),
    ('fc1.weight.table', Fc1w_table, 'Name of the table containing the fc1.weight matrix'),
    ('fc1.bias.table',   Fc1b_table, 'Name of the table containing the fc1.bias vector'),
    ('fc2.weight.table', Fc2w_table, 'Name of the table containing the fc2.weight matrix'),
    ('fc2.bias.table',   Fc2b_table, 'Name of the table containing the fc2.bias vector'),
    ('fc3.weight.table', Fc3w_table, 'Name of the table containing the fc3.weight matrix'),
    ('fc3.bias.table',   Fc3b_table, 'Name of the table containing the fc3.bias vector'),
]

# Insert the header records
deleteRows(DB_path, 'Header')  # delete previous records
insertRecordList(DB_path, insertHeaderRecord, header_records)
records = getRecords(DB_path, 'Header')
for r in records: print(r[:-1])   # print all but description field

(2, 'name', 'MLP-100')
(3, 'architecture', '784-FC:100-FC:100-10')
(4, 'accuracy', 98.36)
(5, 'correct_count', 9836)
(6, 'Hparam.table', 'Hparam_T')
(7, 'fc1.weight.table', 'FC1_Weight_T')
(8, 'fc1.bias.table', 'FC1_Bias_T')
(9, 'fc2.weight.table', 'FC2_Weight_T')
(10, 'fc2.bias.table', 'FC2_Bias_T')
(11, 'fc3.weight.table', 'FC3_Weight_T')
(12, 'fc3.bias.table', 'FC3_Bias_T')


## Write the Hparam table

In [20]:
# Creates the Hparam table
def createHparamTable(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # Create the table
    query_str = f'''CREATE TABLE IF NOT EXISTS {Hparam_table} (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        key TEXT,
                        value NUMERIC,
                        description TEXT
                    )'''
    cursor.execute(query_str)
    # Commit the changes and close the connection
    conn.commit()
    conn.close()


# Create the table and check 
createHparamTable(DB_path)
table_names = getTableNames(DB_path)
print(table_names)

['Header', 'sqlite_sequence', 'Hparam_T']


In [21]:
# Inserts a record into the Hparam table
def insertHparamRecord(cursor, record):
    key, value, description = record  # this serves as a soft check for the record
    # Insert the record into the table
    cursor.execute(f'''INSERT INTO {Hparam_table} (key, value, description)
                      VALUES (?, ?, ?)''', (key, value, description))


# Call the function to insert a record
insertRecordList(DB_path, insertHparamRecord, [('example_key', 123, 'example_description')])
getRecords(DB_path, Hparam_table)

[(1, 'example_key', 123, 'example_description')]

In [22]:
# Insert the Hparam records
print('Hparam from pytorch:', Hparam)
hparam_records = [
    ('input_size', Hparam['input_size'], 'Input size of the MLP'),
    ('num_classes', Hparam['num_classes'], 'Output size of the MLP'),
    
    ('fc1.weight.row', len(fc1_weight_np), 'No. of rows in fc1.weight'),
    ('fc1.weight.col', len(fc1_weight_np[0]), 'No. of columns in fc1.weight'),
    ('fc1.bias.len', len(fc1_bias_np), 'Lengths of the fc1.bias vector'),
    
    ('fc2.weight.row', len(fc2_weight_np), 'No. of rows in fc2.weight'),
    ('fc2.weight.col', len(fc2_weight_np[0]), 'No. of columns in fc2.weight'),
    ('fc2.bias.len', len(fc2_bias_np), 'Lengths of the fc2.bias vector'),
    
    ('fc3.weight.row', len(fc3_weight_np), 'No. of rows in fc3.weight'),
    ('fc3.weight.col', len(fc3_weight_np[0]), 'No. of columns in fc3.weight'),
    ('fc3.bias.len', len(fc3_bias_np), 'Lengths of the fc3.bias vector'),
]


deleteRows(DB_path, Hparam_table)
insertRecordList(DB_path, insertHparamRecord, hparam_records)
getRecords(DB_path, Hparam_table)

Hparam from pytorch: {'input_size': 784, 'num_classes': 10}


[(2, 'input_size', 784, 'Input size of the MLP'),
 (3, 'num_classes', 10, 'Output size of the MLP'),
 (4, 'fc1.weight.row', 100, 'No. of rows in fc1.weight'),
 (5, 'fc1.weight.col', 784, 'No. of columns in fc1.weight'),
 (6, 'fc1.bias.len', 100, 'Lengths of the fc1.bias vector'),
 (7, 'fc2.weight.row', 100, 'No. of rows in fc2.weight'),
 (8, 'fc2.weight.col', 100, 'No. of columns in fc2.weight'),
 (9, 'fc2.bias.len', 100, 'Lengths of the fc2.bias vector'),
 (10, 'fc3.weight.row', 10, 'No. of rows in fc3.weight'),
 (11, 'fc3.weight.col', 100, 'No. of columns in fc3.weight'),
 (12, 'fc3.bias.len', 10, 'Lengths of the fc3.bias vector')]

## Write the weights and biases

In [23]:
# Saves a numpy 2D array as a table in the database.
# Columns: ID, row_no, col_0, col_1, ..., col_n
def createMatrixTable(db_path, table_name, nparray, overwrite=False):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Drop the table if it exists and overwrite requested
    table_exist = existTable(db_path, table_name)
    if overwrite and table_exist:
        print(f'WARN: Overwriting table {table_name}')
        dropTable(db_path, table_name)
        
    # Create the table
    rows, cols = nparray.shape
    column_names = "row_no, " + ", ".join([f"col_{i}" for i in range(cols)])  # Generate the column names string
    cursor.execute(f"CREATE TABLE {table_name} ({column_names})")

    # Insert the array rows into the table
    for i in range(rows):
        vals = f'{i}, ' + ', '.join(map(str, nparray[i]))
        cursor.execute(f"INSERT INTO {table_name} VALUES ({vals})")

    # Commit the changes and close the connection
    conn.commit()
    conn.close()


# test createMatrixTable()
createMatrixTable(DB_path, 'test', fc1_weight_np, overwrite=True)
col_names = getColNames(DB_path, 'test')
records = getRecords(DB_path, 'test')
print('col_names:', col_names[:5], '...', col_names[-5:])
print('records[i]:', records[2][:5], '...')

col_names: ['row_no', 'col_0', 'col_1', 'col_2', 'col_3'] ... ['col_779', 'col_780', 'col_781', 'col_782', 'col_783']
records[i]: (2, -3.39176e-40, 1.06084e-40, 3.82982e-40, 1.488e-40) ...


In [24]:
# Save the weights
createMatrixTable(DB_path, Fc1w_table, fc1_weight_np, overwrite=True)
createMatrixTable(DB_path, Fc2w_table, fc2_weight_np, overwrite=True)
createMatrixTable(DB_path, Fc3w_table, fc3_weight_np, overwrite=True)

# Convert vectors into 2D array for the table
fc1b = np.expand_dims(fc1_bias_np, axis=0)
fc2b = np.expand_dims(fc2_bias_np, axis=0)
fc3b = np.expand_dims(fc3_bias_np, axis=0)
print('fc1b shape:', fc1b.shape)

createMatrixTable(DB_path, Fc1b_table, fc1b, overwrite=True)
createMatrixTable(DB_path, Fc2b_table, fc2b, overwrite=True)
createMatrixTable(DB_path, Fc3b_table, fc3b, overwrite=True)

fc1b shape: (1, 100)


In [25]:
# Drop extra tables
keep_tables = {'sqlite_sequence', 'Header', 'Hparam_T', 
               'FC1_Weight_T', 'FC2_Weight_T', 'FC3_Weight_T', 
               'FC1_Bias_T', 'FC2_Bias_T', 'FC3_Bias_T'}

all_tables = getTableNames(DB_path)
cnt = 0
for name in all_tables:
    if name not in keep_tables:
        dropTable(DB_path, name)
        print(f'WARN: Dropped table {name}')
        cnt += 1
print(f'INFO: {cnt} tables dropped')

all_tables = getTableNames(DB_path)
ipd.display(all_tables)

WARN: Dropped table test
INFO: 1 tables dropped


['Header',
 'sqlite_sequence',
 'Hparam_T',
 'FC1_Weight_T',
 'FC2_Weight_T',
 'FC3_Weight_T',
 'FC1_Bias_T',
 'FC2_Bias_T',
 'FC3_Bias_T']

## Import Saved Model and Validate

In [26]:
# Check the meta tables
rec_list = getRecords(DB_path, 'Header')
print('Header:')
for r in rec_list: print(r[1:-1])

print('')
rec_list = getRecords(DB_path, 'Hparam_T')
print('Hparam:')
for r in rec_list: print(r[1:-1])
    


Header:
('name', 'MLP-100')
('architecture', '784-FC:100-FC:100-10')
('accuracy', 98.36)
('correct_count', 9836)
('Hparam.table', 'Hparam_T')
('fc1.weight.table', 'FC1_Weight_T')
('fc1.bias.table', 'FC1_Bias_T')
('fc2.weight.table', 'FC2_Weight_T')
('fc2.bias.table', 'FC2_Bias_T')
('fc3.weight.table', 'FC3_Weight_T')
('fc3.bias.table', 'FC3_Bias_T')

Hparam:
('input_size', 784)
('num_classes', 10)
('fc1.weight.row', 100)
('fc1.weight.col', 784)
('fc1.bias.len', 100)
('fc2.weight.row', 100)
('fc2.weight.col', 100)
('fc2.bias.len', 100)
('fc3.weight.row', 10)
('fc3.weight.col', 100)
('fc3.bias.len', 10)


In [27]:
# Returns a table saved using createMatrixTable as a list of tuples
def readMatrixTable(db_path, table_name):
    # read the records
    rec_list = getRecords(db_path, table_name)
    # build the matrix
    rec_list.sort()         # sort by row_no (first column)
    matrix = []
    for rec in rec_list:
        matrix.append(rec[1:])  # stripe off the row_no columns
    return matrix


# test this functions
mat1 = np.array(readMatrixTable(DB_path, Fc1w_table))
mat1.shape

(100, 784)

In [28]:
# Returns the weights and biases as a dictionary
def readModelParam(db_path, table_names):
    model_params = {}
    for name in table_names:
        # read the matrix as a list of tuples
        mat = readMatrixTable(db_path, name)
        # Check if it is a matrix or a vector
        if len(mat)==1: is_vector = True
        else: is_vector = False
        # convert to numpy array
        if is_vector: mat = np.array(mat[0])    # make a 1D array for vectors
        else: mat = np.array(mat)
        # save it for returning
        model_params[name] = mat
    return model_params
        

# test this function
param_tables = [
    'FC1_Weight_T',
    'FC2_Weight_T',
    'FC3_Weight_T',
    'FC1_Bias_T',
    'FC2_Bias_T',
    'FC3_Bias_T'
]

model_params = readModelParam(DB_path, param_tables)
for k, v in model_params.items():
    print(f'{k}:', v.shape, v.dtype)

FC1_Weight_T: (100, 784) float64
FC2_Weight_T: (100, 100) float64
FC3_Weight_T: (10, 100) float64
FC1_Bias_T: (100,) float64
FC2_Bias_T: (100,) float64
FC3_Bias_T: (10,) float64


In [29]:
# Compare with original weights
org_params = {
    'FC1_Weight_T': fc1_weight_np,
    'FC2_Weight_T': fc2_weight_np,
    'FC3_Weight_T': fc3_weight_np,
    'FC1_Bias_T': fc1_bias_np,
    'FC2_Bias_T': fc2_bias_np,
    'FC3_Bias_T': fc3_bias_np
}

def compare_model_params(model_params, org_params, tolerance):
    for k in model_params:
        print('\nComparing:', k)
        db_val = model_params[k]
        org_val = org_params[k]
        dmin = np.min(org_val)
        dmax = np.max(org_val)
        print('min:', dmin, '  max:', dmax)
        diff_val = np.max(np.abs(db_val - org_val))   # get the maximum difference
        print('diff:', diff_val)
        assert diff_val <= tolerance   # use manual check
        assert np.allclose(db_val, org_val, rtol=tolerance)  # use numpy built-in check


# compare with tolerance
tolerance = 1e-6
compare_model_params(model_params, org_params, tolerance)


Comparing: FC1_Weight_T
min: -1.2509781   max: 0.66414034
diff: 5.6259155334359434e-08

Comparing: FC2_Weight_T
min: -0.6132809   max: 0.705184
diff: 2.8821563691217023e-08

Comparing: FC3_Weight_T
min: -0.55377436   max: 0.40687668
diff: 1.629867552033204e-08

Comparing: FC1_Bias_T
min: -0.8386414   max: 0.53778327
diff: 2.9688262959126632e-08

Comparing: FC2_Bias_T
min: -0.5823776   max: 1.2394397
diff: 4.467773440097744e-08

Comparing: FC3_Bias_T
min: -0.41325337   max: 0.74696374
diff: 6.000137331430011e-09


# Export the Dataset as sqlite3 DB

In [30]:
# Create the database file
DB2_path = './saved/mnist_test_data.s3db'
createDB(DB2_path, overwrite=True)
!ls -ltr saved/

WARN: Overwriting existing file ./saved/mnist_test_data.s3db
total 156192
-rw-rw-r-- 1 makabir makabir   360783 Jun 20 17:42 trained_mlp100-98.11p.pt
-rw-rw-r-- 1 makabir makabir   360783 Jun 20 18:07 trained_mlp100-98.34p.pt
-rw-rw-r-- 1 makabir makabir   360783 Jun 21 13:55 trained_mlp100.pt
-rw-rw-r-- 1 makabir makabir 70809849 Jun 21 13:55 test_dataset.pt
-rw-rw-r-- 1 makabir makabir   360783 Jun 21 13:57 trained_mlp100-98.36p.pt
-rw-r--r-- 1 makabir makabir    16384 Jun 21 15:02 model.s3db
-rw-r--r-- 1 makabir makabir  1818624 Jun 21 16:12 model-mlp100.s3db
-rw-r--r-- 1 makabir makabir  1818624 Jun 21 19:52 trained_mlp100-98.36p.s3db
-rw-r--r-- 1 makabir makabir 82194432 Jun 21 19:53 mnist_test_data-98.36p.s3db
-rw-r--r-- 1 makabir makabir  1818624 Jun 21 19:58 trained_mlp100.s3db
-rw-r--r-- 1 makabir makabir        0 Jun 21 19:58 mnist_test_data.s3db


## Save the Header table

In [31]:
# Create the table and check 
createHeaderTable(DB2_path)
table_names = getTableNames(DB2_path)
print(table_names)

['Header', 'sqlite_sequence']


In [32]:
# Define the records as a list
Label_table = 'Labels_T'
Dataitem_table = 'DataItems_T'
Feature_table = 'Features_T'

item = DS_loaded['dataset'][0]
Feature_len = len(item[-1])    # last field in the item is the feature_vector

header_records = [
    ('name', 'MNIST-Test', 'Features (flattened images) extracted from the test dataset of MNIST.'),
    ('feature_length', Feature_len, 'The length of the feature. These features can be directly fed to the MLP100 model'),
    ('accuracy', Accuracy, 'Accuracy of the MLP100 model used to generate the "predicted_index" values.'),
    
    ('labels.table',   Label_table, 'Index to label mapping. The model predicts an index, which can be converted to the label using this table'),
    ('dataset.table', Dataitem_table, 'This table serves as the (label, feature) list. The actual features are stored in a separate table.'), 
    ('dataitem.schema', '', 'There are 3 label-related fields in the dataset.table: "label" is the ground-truth, "label_index" is the index into the index-to-label mapping, "predicted_index" is the index predicted by the trained MLP100 model'),
    ('features.table', Feature_table, 'Contains the actual features for the model.'),
]

# Insert the header records
deleteRows(DB2_path, 'Header')  # delete previous records
insertRecordList(DB2_path, insertHeaderRecord, header_records)
records = getRecords(DB2_path, 'Header')
for r in records: print(r[:-1])   # print all but description field

(1, 'name', 'MNIST-Test')
(2, 'feature_length', 784)
(3, 'accuracy', 98.36)
(4, 'labels.table', 'Labels_T')
(5, 'dataset.table', 'DataItems_T')
(6, 'dataitem.schema', '')
(7, 'features.table', 'Features_T')


## Save the data-items and features

In [33]:
# Split the dataitems for DataItem table and Features table
dataitem_records = []
feature_records = []

item = DS_loaded['dataset'][0]
print('item[-1].type:', type(item[-1]))

for item_index, item in enumerate(DS_loaded['dataset']):
    label, label_index, pred_index, feat_len, feat_vec = item   # parse the item
    feat_id = item_index      # use the index in the dataset as the feature ID
    item_rec = [label, label_index, pred_index, feat_id]
    feat_rec = [feat_id] + feat_vec    # feature-record: (feature-id, col_0, col_1, ...)
    dataitem_records.append(item_rec)
    feature_records.append(feat_rec)

# check the records
print('')
print('dataitem_records:', len(dataitem_records), len(dataitem_records[0]))
print('feature_records:', len(feature_records), len(feature_records[0]))

check_index = 100
item = DS_loaded['dataset'][check_index]
assert feature_records[check_index][1:] == item[-1]

item[-1].type: <class 'list'>

dataitem_records: 10000 4
feature_records: 10000 785


### Save Data Items

In [34]:
# Creates the dataset table to save the data-items
def createDataTable(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Define the table name and column names
    table_name = Dataitem_table
    columns = ["id INTEGER PRIMARY KEY AUTOINCREMENT",
               "label TEXT",
               "label_index INTEGER",
               "predicted_index INTEGER",
               "feature_id INTEGER"]

    # CCreate the table
    query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(columns)})"
    cursor.execute(query)
    conn.commit()
    conn.close()


# Create the table
createDataTable(DB2_path)
table_names = getTableNames(DB2_path)
print(table_names)

['Header', 'sqlite_sequence', 'DataItems_T']


In [35]:
# Inserts a record into the Dataset table
def insertDataRecord(cursor, record):
    label, label_index, predicted_index, feature_id = record  # this serves as a soft check for the record
    # Insert the record into the table
    query = f"INSERT INTO {Dataitem_table} (label, label_index, predicted_index, feature_id) VALUES (?, ?, ?, ?)"
    cursor.execute(query, (label, label_index, predicted_index, feature_id))


# Call the function to insert a record
insertRecordList(DB2_path, insertDataRecord, [("Item 1", 1, 2, 3)])
getRecords(DB2_path, Dataitem_table)

[(1, 'Item 1', 1, 2, 3)]

In [36]:
# insert all dataset records
deleteRows(DB2_path, Dataitem_table)   # delete old records
insertRecordList(DB2_path, insertDataRecord, dataitem_records)
rec_list = getRecords(DB2_path, Dataitem_table)
print('rec_list:', len(rec_list), len(rec_list[0]))
print('rec_list[0]', rec_list[0])

rec_list: 10000 5
rec_list[0] (2, '7', 7, 7, 0)


### Save Features

In [37]:
# Creates the features table to save the feature_vectors
# Columns: feature_id, col_0, col_1, ..., col_n
def createFeatureTable(db_path, table_name, feature_list, overwrite=False):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Drop the table if it exists and overwrite requested
    table_exist = existTable(db_path, table_name)
    if overwrite and table_exist:
        print(f'WARN: Overwriting table {table_name}')
        dropTable(db_path, table_name)
        
    # Create the table
    cols = len(feature_list[0]) - 1    # ommitting the feature-id column from count
    column_names = "feature_id, " + ", ".join([f"col_{i}" for i in range(cols)])  # Generate the column names string
    cursor.execute(f"CREATE TABLE {table_name} ({column_names})")

    # Insert the featurs into the table
    for feat_item in tqdm(feature_list):
        feat_id = feat_item[0]
        feat_vec = feat_item[1:]
        vals = f'{feat_id}, ' + ', '.join(map(str, feat_vec))
        cursor.execute(f"INSERT INTO {table_name} VALUES ({vals})")

    # Commit the changes and close the connection
    conn.commit()
    conn.close()


# Create the table
createFeatureTable(DB2_path, Feature_table, feature_records, overwrite=True)
table_names = getTableNames(DB2_path)
print(table_names)

rec_list = getRecords(DB2_path, Feature_table)
print('rec_list:', len(rec_list))

  0%|          | 0/10000 [00:00<?, ?it/s]

['Header', 'sqlite_sequence', 'DataItems_T', 'Features_T']
rec_list: 10000


In [38]:
feat_item = rec_list[0]
print('feat_item:', len(feat_item))
type_count = {}
for d in feat_item: 
    t = type(d)
    if t not in type_count: type_count[t] = 0
    type_count[t] += 1
print(type_count)

feat_item: 785
{<class 'int'>: 1, <class 'float'>: 784}


## Import Saved Dataset and Verify

In [39]:
# Check the meta tables
rec_list = getRecords(DB2_path, 'Header')
print('Header:')
for r in rec_list: print(r[1:-1])

Header:
('name', 'MNIST-Test')
('feature_length', 784)
('accuracy', 98.36)
('labels.table', 'Labels_T')
('dataset.table', 'DataItems_T')
('dataitem.schema', '')
('features.table', 'Features_T')


In [40]:
# Compare with original features and dataitems
def compare_dataset(db_items, org_items, tolerance):
    range_iter = tqdm(range(len(db_items)))
    for i in range_iter:
        #print(i)
        db_rec = db_items[i]
        org_rec = org_items[i]
        # Compare the labels
        db_labels = db_rec[:3]
        org_labels = org_rec[:3]
        #print('db_rec:', db_rec)
        #print('org_rec:', org_rec)
        assert db_labels==org_labels, "Labels mismatch"
        #if i==5: break
        # Check features
        org_feat = np.array(org_rec[-1])
        db_feat = np.array(db_rec[-1])
        assert np.allclose(org_feat, db_feat, tolerance), "Feature vector mismatch"
    print(f'INFO: Compared {(i+1)} records')


# merge the tables to make similar records as in DS_loaded['dataset']
# build a feat_id: feat_vec map for merging.
feat_records = getRecords(DB2_path, Feature_table)
feat_rec_map = {}
for fitem in feat_records:
    feat_id = fitem[0]
    feat_vec = fitem[1:]
    feat_rec_map[feat_id] = feat_vec

# merge feature vectors with dataset items for comparison
data_records = getRecords(DB2_path, Dataitem_table)
db_items = []
for drec in data_records:
    feat_id = drec[-1]
    feat_vec = feat_rec_map[feat_id]
    merged_item = list(drec[1:4]) + [feat_vec]
    merged_item[0] = int(merged_item[0])
    db_items.append(merged_item)
        
# compare with tolerance
tolerance = 1e-6
compare_dataset(db_items, DS_loaded['dataset'], tolerance)

  0%|          | 0/10000 [00:00<?, ?it/s]

INFO: Compared 10000 records


# Concluding Remarks

Now you can use these databases to translate the Numpy model into C implementations. You can also run experiments on fixed-point precisions.